In [1]:
python_code = '''
import random
result = 'heads' if random.randint(0,1) == 0 else 'tails'
print(result)

import time
time.sleep(10)
'''

In [2]:
import kfp
from kfp import dsl

@dsl.pipeline(
    name='Sequential pipeline', 
    description='A pipeline with two sequential steps.'
)
def sequential_pipeline():
    flip = flip_coin_op()
    print_op(flip.output)

def flip_coin_op():
    return dsl.ContainerOp(
        name='Flip coin',
        image='python:alpine3.6',
        command=['sh', '-c'],
        arguments=[f'python -c "{python_code}" | tee /tmp/output'],
        file_outputs={'output': '/tmp/output'}
    )

def print_op(msg):
    return dsl.ContainerOp(
        name='Print',
        image='alpine:3.6',
        command=['echo', msg],
    )

In [3]:
pipelineGzFile = 'simple-flipcoin-01.zip'

kfp.compiler.Compiler().compile(
    sequential_pipeline,
    pipelineGzFile
)

client = kfp.Client()
my_exp = client.create_experiment(
    name='Simple FlipCoin'
)
my_run = client.run_pipeline(
    my_exp.id,
    'Sequential pipeline', 
    pipelineGzFile
)

/opt/conda/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1261: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(
